# How to perform a dark matter analysis with ctools

In this tutorial you will learn how to use ctools to constrain the parameters of a dark-matter model
(mass and annihilation cross section). We will consider as an example the dwarf spheroidal galaxy
Aquarius II. Credits for the notebook:Alejandra Aguirre-Santaella, Tarek Hassan, and Sergio Hernández Cadena

Lets start by importing the required Python modules

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np

import gammalib
import ctools
import cscripts

import time

import sys
import os
from show_residuals import plot_residuals

This notebook was tested using python 3 (more specifically 3.7.6) ctools/gammalib version 1.7.4. If you have errors, please check first if they dissapear when using this version!

In [ ]:
print(ctools.__version__)
print(gammalib.__version__)

## Modelling the source

For simplicity we assume the galaxy is a point source for CTA.

We take the value of the astrophysical J factor from [Pace & Strigari, 2018](https://arxiv.org/abs/1802.06811) (units: $\text{GeV}^2~\text{cm}^{-5}$). Coordinates for the Aquarius II galaxy are taken from [Torealba et al., 2016](https://academic.oup.com/mnras/article/463/1/712/2589989).

In [ ]:
srcname = 'AquariusII'
jfactor = 1.8621e+18

ra      = 338.4813
dec     = -9.3274

The dark-matter annihilation spectrum is modelled using the tables from [Cirelli et al., 2011](http://www.marcocirelli.net/PPPC4DMID.html). The tables were formatted in such a way that they can be ingested by ctools. You can check this [example](https://github.com/sergiohcdna/ctaAnalysis/blob/development/examples/create_dmtable.py) to see how to create the file from the literature material. We provide two files depending if the spectrum include or not electroweak corrections. The example FITS file contains the tables for different annihilation channels, gamma-ray energies between $30~\text{GeV}$ and $100~\text{TeV}$, and dark-matter masses in the range $[0.1,100]~\text{TeV}$. Using the spectrum in this way allow us to only change the flux normalization for other galaxies. See below.

In [ ]:
dmspecEW = 'DMModelAnnihilationAquariusIIEW1.fits'

For our example we will set the DM mass (GeV) and annihilation channel to $\tau$'s as follows.

In [ ]:
mass    = 5.0e+3
channel = 8

For the moment we will set the value of the annihilation cross section ($\langle\sigma_{\chi}v\rangle$ in $\text{cm}^3~\text{s}^{-1}$) based on the upper limits obtained with data from the H.E.S.S. experiment: [Search for dark matter signals towards recently detected DES dwarf galaxy satellites](https://arxiv.org/abs/2008.00688).

In [ ]:
sigmav = 2.7e-22

We can now combine J factor, mass, and annhilihation cross section to derive the overall flux normalization for this model.

In [ ]:
fluxnorm  = jfactor * sigmav / (8*gammalib.pi*mass**2)
fluxnorm *= 1.0e-3

We use the GModelSpectralTable class in GammaLib to manage the spectral model. You can check [gammalib Documentation about Spectral components](http://cta.irap.omp.eu/gammalib/users/user_manual/modules/model/sky/spectral.html) for more information anout GModelSpectralTable. You can also follow the tutorial *How to do advanced model manipulation and fitting in Python* to learn how handle models with GammaLib.

The steps necessary to set up the spectral model are
- load the fits file
- set the value of the mass we will simulate
- set the annihilation channel
- set the normalization of the spectrum

The names of the dark-matter parameters are set within the FITS file in the PARAMETERS extension.

In [ ]:
dmspec = gammalib.GModelSpectralTable()
dmspec.load(dmspecEW)
dmspec['Mass'].value(mass)
dmspec['Channel'].value(channel)
dmspec['Channel'].scale(1)
dmspec['Normalization'].value(fluxnorm)
dmspec['Normalization'].range(0.0, 1.0e+20)

The normalization is a free parameter by default. In this example, we will aditionally set free the mass

In [ ]:
dmspec['Mass'].free()

Now, we can take a look at the spectral information of our source. Note that we will use the spectrum with electroweak corrections.

In [ ]:
print(dmspec)

We then create the spatial model and the source model using the appropriate GammaLib classes.

In [ ]:
spatial = gammalib.GModelSpatialPointSource(ra, dec)

dmmodel = gammalib.GModelSky(spatial, dmspec)
dmmodel.name(srcname)
dmmodel.tscalc(True) # calculate TS for this source

### Questions

1. Mention the features of photon spectra from annihilation of dark matter particles. 
2. Plot the spectrum of the source we will be simulating. Hint: use the dmspec.eval() function, which needs gammalib.GEnergy as input.

## Simulating CTA Observations

### Sky model

We also need a model for the instrumental background, that we will read from the IRFs with a power-law spectral correction.

In [ ]:
# spectral correction
spectral = gammalib.GModelSpectralPlaw(1, 0, gammalib.GEnergy(1, 'TeV'))

# background model
bkgmodel = gammalib.GCTAModelIrfBackground(spectral)
bkgmodel.name('Background')
bkgmodel.instruments('CTA')

We combine the two models in a container and save it to disk for later use.

In [ ]:
inmodel = "inmodel.xml"
models  = gammalib.GModels()
models.append(dmmodel)
models.append(bkgmodel)
models.save(inmodel)

### Exercise: Observation simulation

We simulate an observation of 30 min pointed at Aquarius II using the instrument response functions for CTA South.

In [ ]:
emin  = 0.03 # TeV
emax  = 100. # TeV
caldb = 'prod3b-v2'
irf   = 'South_z40_50h'

obssim = ctools.ctobssim()

### Exercise: creating a skymap

Peek at the simulated events by creating a skymap.

In [ ]:
skymap = ctools.ctskymap(obssim.obs())


## Exercise: Likelihood Analysis

Run the likelihood analysis

In [ ]:
like = ctools.ctlike(obssim.obs())
like.run()

Let's check the TS of the source

In [ ]:
like.obs().models()[srcname].ts()

### Exercise
Generate residual map and spectrum to verify that the fitted model reproduces the data well.

In [ ]:
resspec = cscripts.csresspec(like.obs())


In [ ]:
plot_residuals('resid_spectrum.fits','',0)

### Results of the analysis

So the model we simulated yields a significant detection of the dark-matter signal!

We can check the fitted model to get the value of the mass and compare with the input.

In [ ]:
print(like.obs().models()[srcname])

For the annihilation cross-section $\langle\sigma_{\chi}v\rangle$ we need to take into account the J factor. The easiest way is to derive the annihilation cross-section based on the ratio $\xi$ between the flux measured and the flux expected for the reference cross-section value. We will consider the integrated flux from the minimum energy of the observations up to the dark-matter mass.
The measured annihilation cross-section therefore is:

$\langle\sigma_{\chi}v\rangle_{\text{meas}} = \xi \times \langle\sigma_{\chi}v\rangle_{\text{ref}}$

### Exercise:

1. Did you see anything in the skymap? We actually didn't... Why not? 
2. Can you come up with any way to see the point-like source we are simulating? ctools is detecting it, so it must be somewhere... 

### Exercise:

1. Derive the uncertainty on $\langle\sigma_{\chi}v\rangle$ from the relative uncertainty of the normalization parameter.
2. What is the "observed" value for the mass of the particle?

In [ ]:
# Define energy range where the integrated flux is computed
gemin = gammalib.GEnergy(emin, 'TeV')
gemax = gammalib.GEnergy(mass, 'GeV')
# Get the reference flux
ref_flux = dmmodel.spectral().flux(gemin, gemax)
# Get the measured flux
meas_flux = like.obs().models()[srcname].spectral().flux(gemin, gemax)
# Measured cross-section
print(meas_sigmav)

In [ ]:
print('Annihilation cross-section from observations:\n')
print('\t{:.3e} +/- {:.3e} cm**3/s'.format(meas_sigmav, sigma_v_error))

## Upper limits

Unfortunately the dark-matter signal may not be so strong that CTA can detect it. In this case you will want to set upper limits on the annihilation cross-section. To explore this scenario we will perform a new simulation with different parameters.

In [ ]:
sigmav = 5.0e-24

fluxnorm  = jfactor * sigmav / (8*gammalib.pi*mass**2)
fluxnorm *= 1.0e-3

In [ ]:
models[srcname].spectral()['Normalization'].value(fluxnorm)
models[srcname].spectral()['Mass'].fix()
models.save(inmodel)

### Exercise: Simulating new observations

We regenerate the observations for the new cross-section value. (Again, 30 min)

In [ ]:
obssim = ctools.ctobssim()
obssim.run()

We perform again a likelihood fit and check the TS value.

In [ ]:
like = ctools.ctlike(obssim.obs())
like.run()

In [ ]:
like.obs().models()[srcname].ts()

The small value of TS means that there is no evidence of any dark-matter signal. Therefore we proceed to compute upper limits on the annihilation cross-section.

In order to do so let's fix the mass and calculate an upper limit on the normalization. For example let's consider a mass of 5 TeV.

In [ ]:
print(like.obs().models()[srcname])

### Exercise

We can use ctulimit to compute an upper limit on the source normalization.

Tip: set wisely the energy range for the calculation of the integral flux upper limits.

In [ ]:
ulimit = ctools.ctulimit(obssim.obs())
ulimit.run()

### Exercise
As we did before we can compare the flux upper limit and reference flux to obtain an upper limit on the annhilation cross-section. Derive the UL to the annihilation cross-secion

In [ ]:
ulimit_flux = ulimit.flux_ulimit()
ul_sigmav =

print(ul_sigmav)

### Exercise

1. Compute the UL on the annihilation cross-section using as spectral model the spectrum without electroweak corrections. You need to load the file "DMModelAnnihilationAquariusIIEW0.fits".
    - Hint: You don't need to rerun ctobssim
    - Please be sure to pass the correct number to indicate the same annihilation channel ($\tau$). Check [Cirelli et al., 2011](http://www.marcocirelli.net/PPPC4DMID.html).

To check if there is a real difference between the two ULs, we first need to compute the average UL after perform several realizations (for example ~50).

### Likelihood Profile

In this case we explore a simple model where the parameter of interest impact the value of the overall normalization of the gamma-ray flux. In other models, the parameter of interest change directly the shape of the spectrum, or inclusive if we want to estimate the impact of nuisance paramaters, is good practice to take a look at the likelihood profile. Let's check what the likelihood profile looks like for the normalization (related to the parameter of interest).

First, we define the range we are interested to check:

In [ ]:
norma_r = np.logspace(-28, 6)

Now, we fix the normalization in our model.

In [ ]:
like.obs().models()[srcname].spectral()['Normalization'].fix()

We compute the likelihood for every normalization value and saving to a list (take several minutes...)

In [ ]:
ll_values = []

for norma in norma_r :
    like.obs().models()[srcname].spectral()['Normalization'].value(norma)
    like.run()
    ll = like.opt().value()
    ll_values.append(ll)

Now, let's plot the results

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))

ax.plot(norma_r, ll_values, color=( 0.82 , 0.10 , 0.26 ), lw=2)

ax.set_ylim(1.e+5, 1.e+11)
ax.set_xlim(1.e-28, 1.e-6)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Normalization')
ax.set_ylabel('$\\log{L}$')
